In [ ]:
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = {"username":"baharehgholami","key":"6fc71ac351606882ff7c1146be58f4b6"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d datamunge/sign-language-mnist

 53% 33.0M/62.6M [00:00<00:00, 168MB/s] 
100% 62.6M/62.6M [00:00<00:00, 206MB/s]


In [ ]:
!unzip /content/sign-language-mnist.zip

Archive:  /content/sign-language-mnist.zip
  inflating: amer_sign2.png          
  inflating: amer_sign3.png          
  inflating: american_sign_language.PNG  
  inflating: sign_mnist_test.csv     
  inflating: sign_mnist_test/sign_mnist_test.csv  
  inflating: sign_mnist_train.csv    
  inflating: sign_mnist_train/sign_mnist_train.csv  


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import torch
from torch import nn
from torch.utils.data import Dataset,DataLoader
import torchvision.transforms as transforms 
from sklearn.model_selection import train_test_split
import torch.optim as optim 

In [ ]:
train=pd.read_csv("/content/sign_mnist_train/sign_mnist_train.csv")
test=pd.read_csv("/content/sign_mnist_test/sign_mnist_test.csv")

In [ ]:
len(np.unique(train["label"]))

24

In [ ]:
np.unique(train["label"])

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24])

In [ ]:
torch.from_numpy(train.iloc[:,1:].values[3].reshape(1,28,28)).dtype

torch.int64

In [ ]:
torch.tensor(train.iloc[:,0].values[7]).dtype


torch.int64

In [ ]:
from torch.nn.modules import linear
class Model(nn.Module):
   def __init__(self):
      super(Model,self).__init__()
      self.sequential=nn.Sequential(nn.Conv2d(1,32,(2,2),padding='same',stride=1),
                                    nn.Conv2d(32,64,(2,2),padding="same"),
                                    nn.MaxPool2d(2),
                                    nn.Conv2d(64,128,(3,3),padding='same'),
                                    nn.Conv2d(128,256,(3,3),padding='same'),
                                    nn.MaxPool2d(2),nn.Flatten(),
                                    nn.Linear(7*7*256,800),
                                    nn.Dropout(0.2),
                                    nn.Linear(800,100),
                                    nn.Dropout(0.5),
                                    nn.Linear(100,25),
                                    nn.Softmax())
   def forward(self, x):
       x=self.sequential(x)
       return x

      

In [ ]:
import torch.nn as nn
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(         
            nn.Conv2d(
                in_channels=1,              
                out_channels=16,            
                kernel_size=5,              
                stride=1,                   
                padding=2,                  
            ),                              
            nn.ReLU(),                      
            nn.MaxPool2d(kernel_size=2),    
        )
        self.conv2 = nn.Sequential(         
            nn.Conv2d(16, 32, 5, 1, 2),     
            nn.ReLU(),                      
            nn.MaxPool2d(2),                
        )
        # fully connected layer, output 10 classes
        self.out = nn.Linear(32 * 7 * 7, 25)
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        # flatten the output of conv2 to (batch_size, 32 * 7 * 7)
        x = x.view(x.size(0), -1)       
        output = self.out(x)
        return output  

In [ ]:
CNN()(torch.randn(32,1,28,28)).shape

torch.Size([32, 25])

In [ ]:
CNN()(torch.randn(32,1,28,28)).argmax(axis=1)

tensor([12, 21, 12,  6, 12, 12, 12, 12, 12, 12, 12, 21, 12, 21, 12, 12, 12, 12,
        12, 12, 12, 21, 12,  0, 12, 12, 12, 12, 12, 12, 12, 11])

In [ ]:
from torch.autograd import Variable
dtype = torch.FloatTensor
def train_fn(model,optimizer,dataloader,loss_fn,device,epoch):
  model.to(device) 
  model.train()
  loss_step=0
  Loss_final=0
  total_step=len(dataloader)
  for i,(x,y) in enumerate(dataloader):
    optimizer.zero_grad()
    #print(x.type)
    x=(x.type(dtype)).to(device)
    y=y.to(device)
    #x=x.to(device)
    #y=y.to(device)
    pred=model(x)
    loss_step=loss_fn(pred,y)
    Loss_final+=loss_step.item()
    loss_step.backward()
    optimizer.step()
    if (i+1) % 100 == 0:
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                       .format(epoch + 1, EPOCHS, i + 1, total_step, loss_step.item()))
    

  Loss_final=Loss_final/len(dataloader)
  return Loss_final

def valid_fn(model,optimizer,dataloader,loss_fn,device):
  model.to(device)
  model.eval()
  num_correct=0
  num_samples = 0
  #prediction_valid=[]
  #Loss_val=0
  for x,y in dataloader:
    x=(x.type(dtype)).to(device)
    y=y.to(device)
    with torch.no_grad():
      pred=model(x)
    #loss=loss_fn(pred,y)
    #Loss_val+=loss.item()
    #prediction_valid.append(pred)
    predict_arg=pred.argmax(axis=1)
    num_correct+=(predict_arg==y).sum()
    num_samples+=pred.size(0)
  print(f"Got {num_correct} / {num_samples} with accuracy {float((num_correct)/float(num_samples))*100:.2f}")

In [ ]:
# HyperParameters
DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 25
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-5
NFOLDS = 5
EARLY_STOPPING_STEPS = 10
EARLY_STOP = False

In [ ]:
class image_dataset(Dataset):
  def __init__(self,data,transform=False):
      super().__init__()
      self.data=data
      self.transform=transform

  def __len__(self):
      return self.data.shape[0]

  def __getitem__(self, index):
        features=self.data.iloc[:,1:].values[index].reshape(1,28,28)
        targets=self.data.iloc[:,0].values[index]
        #targets=torch.Tensor(targets)
        if self.transform:
          features=self.standard(features)
          features=torch.from_numpy(features)
        return features, targets
  def standard(self,x): #x:torch.float64
        return ((x-159.29)/48.76)

In [ ]:
trainset,valset=train_test_split(train,test_size=0.1,random_state=66)

trainset2=image_dataset(trainset,transform=True)
train_loader=DataLoader(trainset2,batch_size=32,shuffle=True)

valset2=image_dataset(valset,transform=True)
val_loader=DataLoader(valset2,batch_size=32,shuffle=False)

model=CNN()
loss_fn=nn.CrossEntropyLoss()
DEVICE="cuda" if torch.cuda.is_available() else "cpu"
best_loss=np.inf
steps=0
optimizer=optim.Adam(model.parameters(),lr=LEARNING_RATE,weight_decay=WEIGHT_DECAY)

for epoch in range(EPOCHS):
  loss=train_fn(model=model,dataloader=train_loader,loss_fn=loss_fn,optimizer=optimizer,device=DEVICE,epoch=epoch)
  print(f'train loss at epoch {epoch} is {loss}')
  if loss<best_loss:
    best_loss=loss
    torch.save(model.state_dict(),"model.pth")
  elif EARLY_STOP == False:
    steps+=1
    if (steps>EARLY_STOPPING_STEPS):
      break

model=CNN()
model.load_state_dict(torch.load("model.pth"))
model.eval()
valid_fn(model,optimizer,val_loader,loss_fn,DEVICE)

Epoch [1/25], Step [100/773], Loss: 0.8359
Epoch [1/25], Step [200/773], Loss: 0.4561
Epoch [1/25], Step [300/773], Loss: 0.2333
Epoch [1/25], Step [400/773], Loss: 0.0485
Epoch [1/25], Step [500/773], Loss: 0.0354
Epoch [1/25], Step [600/773], Loss: 0.0313
Epoch [1/25], Step [700/773], Loss: 0.0153
train loss at epoch 0 is 0.3625246050016398
Epoch [2/25], Step [100/773], Loss: 0.0015
Epoch [2/25], Step [200/773], Loss: 0.0022
Epoch [2/25], Step [300/773], Loss: 0.0013
Epoch [2/25], Step [400/773], Loss: 0.1378
Epoch [2/25], Step [500/773], Loss: 0.2448
Epoch [2/25], Step [600/773], Loss: 0.0032
Epoch [2/25], Step [700/773], Loss: 0.0016
train loss at epoch 1 is 0.018697182435572765
Epoch [3/25], Step [100/773], Loss: 0.0015
Epoch [3/25], Step [200/773], Loss: 0.0006
Epoch [3/25], Step [300/773], Loss: 0.0004
Epoch [3/25], Step [400/773], Loss: 0.0003
Epoch [3/25], Step [500/773], Loss: 0.0006
Epoch [3/25], Step [600/773], Loss: 0.0001
Epoch [3/25], Step [700/773], Loss: 0.0005
train l

In [ ]:
model=CNN()
model.load_state_dict(torch.load("model.pth"))
model.eval()
valid_fn(model,optimizer,val_loader,loss_fn,DEVICE)

Got 2746 / 2746 with accuracy 100.00


In [ ]:
test["id"]=[i for i in range(test.shape[0])]

In [ ]:
test

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784,id
0,6,149,149,150,150,150,151,151,150,151,...,148,127,89,82,96,106,112,120,107,0
1,5,126,128,131,132,133,134,135,135,136,...,104,194,183,186,184,184,184,182,180,1
2,10,85,88,92,96,105,123,135,143,147,...,166,242,227,230,227,226,225,224,222,2
3,0,203,205,207,206,207,209,210,209,210,...,248,247,248,253,236,230,240,253,255,3
4,3,188,191,193,195,199,201,202,203,203,...,40,64,48,29,46,49,46,46,53,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7167,1,135,119,108,102,105,99,61,103,121,...,112,116,114,118,180,184,176,167,163,7167
7168,12,157,159,161,164,166,166,171,174,175,...,213,213,214,213,211,210,210,209,208,7168
7169,2,190,191,190,191,190,190,192,192,191,...,215,213,214,214,213,210,211,209,208,7169
7170,4,201,205,208,209,214,216,218,223,226,...,169,255,255,237,113,91,67,70,63,7170


In [ ]:
class test_dataset(Dataset):
  def __init__(self,data,transform=False):
      super().__init__()
      self.data=data
      self.transform=transform

  def __len__(self):
      return self.data.shape[0]

  def __getitem__(self, index):
        features=self.data.iloc[:,1:-1].values[index].reshape(1,28,28)
       
        if self.transform:
          features=self.standard(features)
          features=torch.from_numpy(features)
        return features
  def standard(self,x): #x:torch.float64
        return ((x-159.29)/48.76)

In [ ]:
print(test.index)

RangeIndex(start=0, stop=7172, step=1)


In [ ]:

test_=test_dataset(test,transform=True)
test_loader=DataLoader(test_,batch_size=32,shuffle=False)

model=CNN()
model.load_state_dict(torch.load("model.pth"))
prediction=[]
prediction_ar=np.zeros((test.shape[0],1))
for x in test_loader:
  model.to(DEVICE)
  model.eval()
  x=(x.type(dtype)).to(DEVICE)
  pred=model(x)
  prediction.append(pred.detach().numpy())
prediction=np.concatenate(prediction)
prediction_ar=prediction



In [ ]:
m=torch.nn.Softmax(dim=1)
p=m(torch.from_numpy(prediction_ar)).argmax(dim=1)
p.to('cpu').numpy()
test["pred"]=p.to('cpu').numpy()

In [ ]:
test

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784,id,pred
0,6,149,149,150,150,150,151,151,150,151,...,127,89,82,96,106,112,120,107,0,6
1,5,126,128,131,132,133,134,135,135,136,...,194,183,186,184,184,184,182,180,1,5
2,10,85,88,92,96,105,123,135,143,147,...,242,227,230,227,226,225,224,222,2,10
3,0,203,205,207,206,207,209,210,209,210,...,247,248,253,236,230,240,253,255,3,0
4,3,188,191,193,195,199,201,202,203,203,...,64,48,29,46,49,46,46,53,4,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7167,1,135,119,108,102,105,99,61,103,121,...,116,114,118,180,184,176,167,163,7167,1
7168,12,157,159,161,164,166,166,171,174,175,...,213,214,213,211,210,210,209,208,7168,12
7169,2,190,191,190,191,190,190,192,192,191,...,213,214,214,213,210,211,209,208,7169,2
7170,4,201,205,208,209,214,216,218,223,226,...,255,255,237,113,91,67,70,63,7170,4


In [ ]:
print((test['label']==test['pred']).sum())
print(test.shape[0])
acc=((test['label']==test['pred']).sum())/test.shape[0]
acc

6685
7172


0.9320970440602342